In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
import math

In [49]:
FHS_data = pd.read_excel('Framingham_data.xlsx')
FHS_data.head()

,sex,sbp,dbp,scl,chdfate,followup,age,bmi,month,id
0,1,120,80,267.0,1,18,55,25.000000,8,2642
1,1,130,78,192.0,1,35,53,28.400000,12,4627
2,1,144,90,207.0,1,109,61,25.100000,8,2568
3,1,92,66,231.0,1,147,48,26.200001,11,4192
4,1,162,98,271.0,1,169,39,28.400000,11,3977


In [106]:
def mean_confidence_interval(data, confidence=0.95):
    n = data.size
    print(f'Data size (n): {n}')
    mean = data.mean()
    print(f'Mean: {mean}')
    # Standard error calculated via sigma / sqrt(n)
    std_error = data.std() / math.sqrt(n)
#     print(data.sem())
    print(f'Standard error of the mean: {std_error}')
    t_score = stat.t.ppf(((1 + confidence) / 2), df=(n-1))
#     print(stat.norm.ppf(.975))
    print(f't-score: {t_score}')
    lower = mean - (std_error * t_score)
    upper = mean + (std_error * t_score)
    return lower, mean, upper

In [108]:
def proportion_confidence_interval(data, confidence=0.95):

SyntaxError: unexpected EOF while parsing (<ipython-input-108-af12094b96dc>, line 1)

In [109]:
def mean_hypothesis_test():

SyntaxError: unexpected EOF while parsing (<ipython-input-109-c456c83887c3>, line 1)

In [110]:
def proportion_hypothesis_test():

SyntaxError: unexpected EOF while parsing (<ipython-input-110-035b2877abd7>, line 1)

## Problem 1

### Part A

### Part B

## Problem 2

### Part A

In [111]:
DBP_no_CHD = FHS_data.where(FHS_data['chdfate'] == 0)['dbp'].dropna()
interval_DBP_no_CHD = mean_confidence_interval(data=DBP_no_CHD)
print(f'\nWe are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals NOT diagnosed with CHD')
print(f'falls within the interval of {interval_DBP_no_CHD[0]:.2f} mmHg to {interval_DBP_no_CHD[-1]:.2f} mmHg.')

Data size (n): 3226
Mean: 81.03099814011159
Standard error of the mean: 0.21531560267883296
t-score: 1.960699843972443

We are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals NOT diagnosed with CHD
falls within the interval of 80.61 mmHg to 81.45 mmHg.


In [112]:
DBP_has_CHD = FHS_data.where(FHS_data['chdfate'] == 1)['dbp'].dropna()
interval_DBP_CHD = mean_confidence_interval(data=DBP_has_CHD)
print(f'\nWe are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals diagnosed with CHD')
print(f'falls within the interval of {interval_DBP_CHD[0]:.2f} mmHg to {interval_DBP_CHD[-1]:.2f} mmHg.')

Data size (n): 1473
Mean: 85.8499660556687
Standard error of the mean: 0.34386798326133367
t-score: 1.9615768852660422

We are 95% confident that the population mean of diastolic blood pressure (dbp) for individuals diagnosed with CHD
falls within the interval of 85.18 mmHg to 86.52 mmHg.


### Part B

## Problem 3

### Part A

### Part B